## House Prices 

In this competition, we are trying to figure out the prices for which a bunch of houses have been sold.
A lot of these features can be used out-of-the-box, so let's check what we have.

In [167]:
import pandas as pd
import numpy as np

# Checking The Data

Let's begin by first loading our training and test set and taking a look at them.

In [168]:
# Id column won't be useful for building the model,
# so let's make it the index of the dataframe
train = pd.read_csv("input/train.csv", index_col=0)
test = pd.read_csv("input/test.csv", index_col=0)

In [169]:
train.shape, test.shape

((1460, 80), (1459, 79))

In [170]:
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000



Our training set has 1460 and 1459 houses on our training and test set, respectively. We have a lot of categorial features in this dataset, so it's interesting to break them into dummy variables. To make it easier for us to create and manipulate these variables, let's join the training and test set, so that both will end up with the same columns. We can split them back after we've done all the data cleaning.


In [171]:
# let's remove sale prices of our train before joining, and save them for later use
Y_train = np.log(train.pop('SalePrice'))

all_df = pd.concat((train, test), axis=0)
all_df.shape

(2919, 79)

Now that we have everything together, we need to generate dummy variables for our categorical features. Pandas can help us with the get_dummy_variable function. It generates dummy variables for a series or an entire dataframe. 

In [172]:
all_df = pd.get_dummies(all_df)

# let's check if everything went alright
all_df.head(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65,8450,7,5,2003,2003,196,706,0,...,0,0,0,1,0,0,0,0,1,0
2,20,80,9600,6,8,1976,1976,0,978,0,...,0,0,0,1,0,0,0,0,1,0
3,60,68,11250,7,5,2001,2002,162,486,0,...,0,0,0,1,0,0,0,0,1,0
4,70,60,9550,7,5,1915,1970,0,216,0,...,0,0,0,1,1,0,0,0,0,0
5,60,84,14260,8,5,2000,2000,350,655,0,...,0,0,0,1,0,0,0,0,1,0



Column order looks a little messy. They are in alphabetical order now, but that shouldn't be a problem. But we do have a lot of columns, which is possibly a bigger deal to be worried with.

Now that we have our dummy variables, we should check for NaN values in our columns. For the categorical features, pandas' get_dummy_variables takes care of this for us. For example, for a feature X which could get values a, b or c, if an individual has NaN for X, all 3 resulting columns (X_a, X_b, X_c) will be assigned 0. (Note: it can be useful to create a new column for missing values, e.g., X_missing, because not being assigned to any label could be informative; we should get back at this later)

Since there's no missing values for our categorical features, we just need to check the numerical ones.


In [173]:
all_df.isnull().sum().sort_values(ascending=False).head(15)

LotFrontage          486
GarageYrBlt          159
MasVnrArea            23
BsmtFullBath           2
BsmtHalfBath           2
BsmtFinSF1             1
BsmtFinSF2             1
BsmtUnfSF              1
TotalBsmtSF            1
GarageArea             1
GarageCars             1
Condition1_RRNe        0
Condition1_RRNn        0
Condition2_Artery      0
Condition1_RRAn        0
dtype: int64

We need to take care of these missing values. For now, let's just assign the mean for all missing values. There's probably better ways to deal with that, but let's just do the easy way now.

In [184]:
mean_columns = all_df.mean()
all_df = all_df.fillna(mean_columns)

all_df.isnull().sum().sort_values(ascending=False).head(10)

SaleCondition_Partial    0
SaleCondition_Normal     0
Condition1_PosA          0
Condition1_PosN          0
Condition1_RRAe          0
Condition1_RRAn          0
Condition1_RRNe          0
Condition1_RRNn          0
Condition2_Artery        0
Condition2_Feedr         0
dtype: int64

And this is all the data preparation we need. There's still a lot of things that could be done, but let just proceed for building the model and checking how well we can do with this.


# Building The Model

First thing we want to do is split our dataframe back to train and test set.

In [201]:
dummy_train_df = all_df.loc[train.index]
dummy_test_df = all_df.loc[test.index]

dummy_train_df.shape, dummy_test_df.shape

((1460, 288), (1459, 288))

## Random Forest Regressor

In [202]:
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.cross_validation import cross_val_score

all_features = list(train.describe().columns.values)

In [205]:
alg = RandomForestRegressor(random_state=1, min_samples_split=3, n_estimators=100, max_depth=9)
test_score = np.sqrt(-cross_val_score(alg, dummy_train_df[all_features], Y_train, cv=5,  scoring='mean_squared_error'))
print(np.mean(test_score))

0.148814859952


## Gradient Boosting Regressor

In [204]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
alg = GradientBoostingRegressor()